# <center>Heat Equation</center>

In [ ]:
%%writefile modules/io.py

import numpy
import matplotlib.pyplot


def initialise_field(filename):
    field = numpy.loadtxt(filename)
    return field


def write_field(field, step):
    matplotlib.pyplot.gca().clear()
    matplotlib.pyplot.imshow(field)
    matplotlib.pyplot.axis('off')
    matplotlib.pyplot.savefig('output/bottle_{0:04d}.png'.format(step))


def to_text_file(field, filename):
    numpy.savetxt(filename, field, fmt='%1.1f')

In [ ]:
%%writefile modules/evolve.py

def evolve(u, u_previous, a, dt, dx2, dy2):

    del_sqrd_u = (u_previous[:-2, 1:-1] - 2 * u_previous[1:-1, 1:-1] +
                  u_previous[2:, 1:-1]) / dx2 + (u_previous[1:-1, :-2] -
                                                 2 * u_previous[1:-1, 1:-1] +
                                                 u_previous[1:-1, 2:]) / dy2

    u[1:-1, 1:-1] = u_previous[1:-1, 1:-1] + dt * a * del_sqrd_u

    u_previous[:, :] = u[:, :]

In [ ]:
%%writefile modules/comms.py

#def halo_exchange():

#def distribute_subfields():

#def gather_subfields():

In [ ]:
%%writefile main.py

import matplotlib.pyplot
import time
import numpy
from mpi4py import MPI
from mpi4py.MPI import Request
from modules import evolve, io, comms

# Set the colormap
matplotlib.pyplot.rcParams['image.cmap'] = 'BrBG'

# Basic parameters
a = 0.5  # Diffusion constant
timesteps = 1000  # Number of time-steps to evolve system
image_interval = 4000  # Write frequency for png files

# Grid spacings
dx = 0.01
dy = 0.01
dx2 = dx**2
dy2 = dy**2

# For stability, this is the largest possible size of the time-step:
dt = dx2 * dy2 / (2 * a * (dx2 + dy2))


def iterate(field, field0, timesteps, image_interval):
    for i in range(1, timesteps + 1):
        evolve.evolve(field, field0, a, dt, dx2, dy2)


def main():
    print('Initialising field...')
    field = io.initialise_field('data/bottle.dat')
    
    io.write_field(field, 0)
    print('Initial field written to output/bottle_0000.png.')

    field0 = field[:, :]

    print('Simulating...')
    t0 = time.time()
    iterate(field, field0, timesteps, image_interval)
    t1 = time.time()
    print('Simulation complete.')
    io.write_field(field, timesteps)
    print('Final field written to output/bottle_{}.png.'.format(timesteps))
    print("Running Time: {0}".format(t1 - t0))


main()

In [ ]:
!mkdir -p output

In [ ]:
!python3 main.py

In [ ]:
import matplotlib.image as img
import matplotlib.pyplot as plt

%matplotlib inline

initial_bottle = img.imread('output/bottle_0000.png')
final_bottle = img.imread('output/bottle_1000.png')

fig=plt.figure(figsize=(16, 16))

fig.add_subplot(1, 2, 1)
plt.gca().set_title('Initial Configuration')
plt.imshow(initial_bottle)
fig.add_subplot(1, 2, 2)
plt.gca().set_title('Final Configuration')
plt.imshow(final_bottle)

plt.show()